<a href="https://colab.research.google.com/github/MunibUrRehmanMemon/Email-Agent/blob/main/Email_Agent_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AI Email Agent with Gemma 3n E4B

This notebook creates an automated email agent that:
1. Scrapes company information and extracts email addresses
2. Uses **Gemma 3n E4B** via Hugging Face to generate personalized emails
3. Sends emails via Gmail API

## Features:
- ✅ Company website scraping with email extraction
- ✅ AI-powered email generation using Gemma 3n E4B
- ✅ Gmail API integration for sending emails
- ✅ Rate limiting and ethical scraping practices
- ✅ GPU acceleration support in Colab

---

## 📦 Installation and Setup

In [ ]:
# Install required packages
!pip install beautifulsoup4 requests fake-useragent tldextract --quiet
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client --quiet
!pip install transformers torch accelerate --quiet
!pip install huggingface_hub --quiet

In [ ]:
# Import required libraries
import os
import re
import time
import random
import logging
import base64
import tldextract
from datetime import datetime
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from urllib.parse import urljoin, urlparse
from urllib.robotparser import RobotFileParser

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

# Gmail API imports
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
from google.colab import auth
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Hugging Face and Transformers imports
import torch # Add this line
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from google.colab import userdata

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 🔑 API Key Setup

In [ ]:
# Get Hugging Face API key from Colab secrets
try:
    HF_TOKEN = userdata.get('HF_token')
    login(token=HF_TOKEN)
    print("✅ Hugging Face authentication successful!")
except Exception as e:
    print(f"❌ Error with Hugging Face authentication: {e}")
    print("Please add your Hugging Face token to Colab secrets with key 'HF_TOKEN'")
    HF_TOKEN = None

✅ Hugging Face authentication successful!


## 📊 Data Classes and Configuration

In [ ]:
@dataclass
class PersonalProfile:
    """Personal profile for email personalization"""
    name: str
    email: str
    linkedin: str
    github: str
    skills: List[str]
    projects: List[str]
    achievements: List[str]
    goal: str

    def skills_str(self) -> str:
        return ", ".join(self.skills)

    def projects_str(self) -> str:
        return "; ".join(self.projects)

    def achievements_str(self) -> str:
        return "; ".join(self.achievements)

@dataclass
class CompanyInfo:
    """Company information extracted from website"""
    name: str
    url: str
    description: str = ""
    emails: List[str] = field(default_factory=list)
    industry: str = ""
    recent_news: str = ""
    keywords: List[str] = field(default_factory=list)

print("✅ Data classes defined successfully!")

✅ Data classes defined successfully!


## 🕷️ Web Scraper Class

In [ ]:
class WebScraper:
    """Enhanced web scraper for company information"""

    def __init__(self):
        self.session = requests.Session()
        self.ua = UserAgent()
        self.last_request_time = 0
        self.min_delay = 2  # Minimum delay between requests

    def _get_headers(self) -> Dict[str, str]:
        """Generate realistic headers for web scraping"""
        return {
            'User-Agent': self.ua.random,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

    def _respect_rate_limit(self):
        """Implement rate limiting"""
        current_time = time.time()
        time_since_last = current_time - self.last_request_time
        if time_since_last < self.min_delay:
            sleep_time = self.min_delay - time_since_last
            time.sleep(sleep_time)
        self.last_request_time = time.time()

    def _extract_emails(self, text: str, domain: str) -> List[str]:
        """Extract email addresses from text and generate common ones"""
        # Find emails in text
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails = re.findall(email_pattern, text)

        # Add common email patterns for this domain
        common_prefixes = ['careers', 'jobs', 'hr', 'info', 'contact', 'hello', 'team', 'hiring']
        domain_emails = [f"{prefix}@{domain}" for prefix in common_prefixes]

        # Combine and deduplicate
        all_emails = list(set(emails + domain_emails))

        # Filter out obviously fake emails
        filtered_emails = []
        for email in all_emails:
            if not any(fake in email.lower() for fake in ['example', 'test', 'sample', 'fake', 'noreply']):
                filtered_emails.append(email)

        return filtered_emails[:5]  # Return top 5 emails

    def scrape_company_info(self, company_name: str, url: str) -> CompanyInfo:
        """Scrape comprehensive company information"""
        logger.info(f"Scraping {company_name} from {url}")

        # Rate limiting
        self._respect_rate_limit()

        try:
            # Extract domain for email generation
            extracted = tldextract.extract(url)
            domain = f"{extracted.domain}.{extracted.suffix}"

            # Try multiple approaches if main URL fails
            urls_to_try = [url]

            # Add alternative URLs for better scraping
            base_domain = f"https://{domain}"
            if url != base_domain:
                urls_to_try.append(base_domain)

            # Add about/company pages
            alternative_paths = ['/about', '/company', '/team', '/contact', '/careers']
            for path in alternative_paths:
                alt_url = urljoin(base_domain, path)
                if alt_url not in urls_to_try:
                    urls_to_try.append(alt_url)

            description = ""
            full_text = ""
            industry_info = ""

            # Try scraping multiple URLs
            for attempt_url in urls_to_try[:4]:  # Limit to 4 attempts
                try:
                    logger.info(f"Attempting to scrape: {attempt_url}")
                    response = self.session.get(attempt_url, headers=self._get_headers(), timeout=15)

                    if response.status_code == 200:
                        soup = BeautifulSoup(response.text, 'html.parser')

                        # Remove script and style elements
                        for script in soup(["script", "style"]):
                            script.decompose()

                        # Extract text content
                        text_content = soup.get_text()

                        # Clean up text
                        lines = (line.strip() for line in text_content.splitlines())
                        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                        clean_text = ' '.join(chunk for chunk in chunks if chunk)

                        if len(clean_text) > len(full_text):
                            full_text = clean_text

                        # Extract company description
                        desc_selectors = [
                            'meta[name="description"]',
                            '.about, .description, .intro',
                            'h1, h2, h3',
                            'p'
                        ]

                        for selector in desc_selectors:
                            elements = soup.select(selector)
                            if elements:
                                if selector.startswith('meta'):
                                    text = elements[0].get('content', '')
                                else:
                                    text = ' '.join([elem.get_text().strip() for elem in elements[:3]])

                                if len(text) > len(description):
                                    description = text[:800]
                                break

                        # Extract industry keywords
                        industry_keywords = []
                        for keyword in ['AI', 'technology', 'software', 'research', 'startup', 'innovation']:
                            if keyword.lower() in clean_text.lower():
                                industry_keywords.append(keyword)

                        if industry_keywords:
                            industry_info = ', '.join(industry_keywords[:3])

                        # If we got good content, break
                        if len(description) > 100:
                            break

                    # Small delay between attempts
                    time.sleep(1)

                except Exception as e:
                    logger.warning(f"Failed to scrape {attempt_url}: {e}")
                    continue

            # Extract emails from the best content we found
            emails = self._extract_emails(full_text, domain)

            # If no description found, create a basic one
            if not description:
                description = f"{company_name} is a technology company working in software development and innovation."

            return CompanyInfo(
                name=company_name,
                url=url,
                description=description,
                emails=emails,
                industry=industry_info,
                recent_news="",
                keywords=industry_info.split(', ') if industry_info else []
            )

        except Exception as e:
            logger.error(f"Error scraping {url}: {e}")
            # Still return something useful with generated emails
            extracted = tldextract.extract(url)
            domain = f"{extracted.domain}.{extracted.suffix}"
            fallback_emails = self._extract_emails("", domain)

            return CompanyInfo(
                name=company_name,
                url=url,
                description=f"{company_name} is a technology company. Visit {url} for more information.",
                emails=fallback_emails,
                industry="Technology",
                keywords=["Technology"]
            )

print("✅ WebScraper class defined successfully!")

✅ WebScraper class defined successfully!


## 🤖 Gemma 3n Email Generator

In [ ]:
class GemmaEmailGenerator:
    """AI-powered email generator using Gemma 3n E4B via Hugging Face"""

    def __init__(self):
        self.model_name = "google/gemma-3n-e4b-it"
        self.tokenizer = None
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

    def load_model(self):
        """Load Gemma 3n E4B model and tokenizer"""
        try:
            print("🔄 Loading Gemma 3n E4B model...")

            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            # Load model with appropriate settings for Colab
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device.type == "cuda" else torch.float32,
                device_map="auto" if self.device.type == "cuda" else None,
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            if self.device.type == "cpu":
                self.model = self.model.to(self.device)

            print("✅ Gemma 3n E4B model loaded successfully!")
            return True

        except Exception as e:
            print(f"❌ Error loading Gemma 3n E4B model: {e}")
            return False

    def generate_personalized_email(self, profile: PersonalProfile, company: CompanyInfo) -> str:
        """Generate a personalized email using Gemma 3n E4B"""

        if self.model is None or self.tokenizer is None:
            if not self.load_model():
                return self._generate_fallback_email(profile, company)

        # Create a sophisticated prompt for Gemma 3n
        prompt = f"""<bos><start_of_turn>user
You are an expert career coach specializing in AI/tech industry networking. Create a highly personalized, professional email.

SENDER PROFILE:
- Name: {profile.name}
- Skills: {profile.skills_str()}
- Key Projects: {profile.projects_str()}
- Achievements: {profile.achievements_str()}
- Career Goal: {profile.goal}

COMPANY CONTEXT:
- Company: {company.name}
- Website: {company.url}
- About: {company.description[:500]}
- Industry Focus: {company.industry}

EMAIL REQUIREMENTS:
1. Professional yet conversational tone
2. Demonstrate genuine research about the company
3. Highlight 2-3 most relevant skills/projects that align with their work
4. Show enthusiasm for their mission/products
5. Request a brief informational chat (not directly asking for a job)
6. Keep it concise (under 200 words)
7. Include appropriate subject line

AVOID:
- Generic templates
- Desperate tone
- Overselling
- Too much technical jargon

Return ONLY the email in this format:
Subject: [subject line]

[email body]

Best regards,
{profile.name}
<end_of_turn>
<start_of_turn>model
"""

        try:
            # Tokenize the prompt
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=400,
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            # Decode the response
            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the model's response (after the last <start_of_turn>model)
            if "<start_of_turn>model" in generated_text:
                email_content = generated_text.split("<start_of_turn>model")[-1].strip()
            else:
                email_content = generated_text[len(prompt):].strip()

            # Clean up the response
            email_content = email_content.replace("<end_of_turn>", "").strip()

            return email_content if email_content else self._generate_fallback_email(profile, company)

        except Exception as e:
            logger.error(f"Email generation error: {e}")
            return self._generate_fallback_email(profile, company)

    def _generate_fallback_email(self, profile: PersonalProfile, company: CompanyInfo) -> str:
        """Generate a basic email template if AI generation fails"""
        return f"""Subject: AI Engineer interested in {company.name}'s innovative work

Dear {company.name} Team,

I'm {profile.name}, an AI engineer with expertise in {', '.join(profile.skills[:3])}.

I've been following {company.name}'s work and am impressed by your innovative approach. My recent projects, including {profile.projects[0] if profile.projects else 'various AI applications'}, align well with your mission.

I'd love to connect for a brief chat about potential opportunities or simply to learn more about your team's exciting work.

Best regards,
{profile.name}
{profile.email}
{profile.linkedin}"""

print("✅ GemmaEmailGenerator class defined successfully!")

✅ GemmaEmailGenerator class defined successfully!


## 📧 Gmail Email Sender

In [ ]:
class EmailSender:
    """Gmail API email sender with enhanced security"""

    def __init__(self, credentials_path: str = None):
        self.credentials_path = credentials_path
        self.token_path = "/content/token.json"
        self.scopes = ['https://www.googleapis.com/auth/gmail.send']
        self.service = None
        self.sent_emails = []

    def authenticate(self):
        """Authenticate with Gmail API"""
        logger.info("Authenticating with Gmail...")

        # Use Google Colab's built-in auth
        auth.authenticate_user()

        creds = None
        if os.path.exists(self.token_path):
            creds = Credentials.from_authorized_user_file(self.token_path, self.scopes)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                if not self.credentials_path or not os.path.exists(self.credentials_path):
                    logger.error("Gmail credentials file not found. Please upload your OAuth2 credentials.")
                    return False

                flow = Flow.from_client_secrets_file(
                    self.credentials_path,
                    scopes=self.scopes,
                    redirect_uri='urn:ietf:wg:oauth:2.0:oob'
                )

                auth_url, _ = flow.authorization_url(prompt='consent')
                print(f"\n🔑 Please visit this URL to authorize the application:\n{auth_url}\n")
                code = input("📥 Enter the authorization code: ")

                flow.fetch_token(code=code)
                creds = flow.credentials

            # Save credentials for next run
            with open(self.token_path, 'w') as token:
                token.write(creds.to_json())

        self.service = build('gmail', 'v1', credentials=creds)
        logger.info("✅ Gmail authentication successful")
        return True

    def send_email(self, to_email: str, subject: str, body: str, from_name: str = None) -> bool:
        """Send an email via Gmail API"""
        if not self.service:
            logger.error("Gmail service not authenticated")
            return False

        try:
            # Create message
            message = MIMEMultipart()
            message['to'] = to_email
            message['subject'] = subject
            if from_name:
                message['from'] = from_name

            # Add body
            message.attach(MIMEText(body, 'plain'))

            # Encode message
            raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

            # Send email
            send_result = self.service.users().messages().send(
                userId="me",
                body={'raw': raw_message}
            ).execute()

            # Track sent email
            self.sent_emails.append({
                'to': to_email,
                'subject': subject,
                'sent_at': datetime.now().isoformat(),
                'message_id': send_result.get('id')
            })

            logger.info(f"✅ Email sent to {to_email}")
            return True

        except Exception as e:
            logger.error(f"❌ Failed to send email to {to_email}: {e}")
            return False

print("✅ EmailSender class defined successfully!")

✅ EmailSender class defined successfully!


## 🤖 Main Email Agent

In [ ]:
class EmailAgent:
    """Main email automation agent with Gemma 3n integration"""

    def __init__(self, profile: PersonalProfile, gmail_credentials_path: str = None):
        self.profile = profile
        self.scraper = WebScraper()
        self.email_generator = GemmaEmailGenerator()
        self.email_sender = EmailSender(gmail_credentials_path)
        self.results = []

    def process_companies(self, companies: Dict[str, str], send_emails: bool = False) -> List[Dict]:
        """Process a list of companies and optionally send emails"""

        if send_emails and not self.email_sender.authenticate():
            logger.error("Email authentication failed. Running in preview mode only.")
            send_emails = False

        logger.info(f"Processing {len(companies)} companies...")

        for company_name, company_url in companies.items():
            try:
                logger.info(f"\n--- Processing {company_name} ---")

                # Scrape company information
                company_info = self.scraper.scrape_company_info(company_name, company_url)

                # Generate personalized email using Gemma 3n
                email_content = self.email_generator.generate_personalized_email(
                    self.profile, company_info
                )

                # Extract subject and body
                lines = email_content.split('\n')
                subject_line = lines[0].replace('Subject:', '').strip() if lines[0].startswith('Subject:') else f"Regarding opportunities at {company_name}"
                email_body = '\n'.join(lines[1:]).strip()

                result = {
                    'company': company_name,
                    'url': company_url,
                    'emails_found': company_info.emails,
                    'subject': subject_line,
                    'email_body': email_body,
                    'sent': False,
                    'sent_to': [],
                    'company_keywords': company_info.keywords
                }

                # Display generated email
                print(f"\n📧 Generated Email for {company_name}:")
                print("=" * 60)
                print(f"Company Keywords: {', '.join(company_info.keywords)}")
                print(f"Found Emails: {', '.join(company_info.emails[:3])}")
                print(f"Subject: {subject_line}")
                print("\nEmail Body:")
                print(email_body)
                print("=" * 60)

                self.results.append(result)

                # Delay between companies
                time.sleep(random.uniform(2, 4))

            except Exception as e:
                logger.error(f"Error processing {company_name}: {e}")
                continue

        return self.results

    def send_generated_emails(self) -> Dict[str, int]:
        """Send all generated emails"""
        if not self.results:
            print("❌ No emails to send. Please run process_companies first.")
            return {'sent': 0, 'failed': 0}

        if not self.email_sender.service:
            if not self.email_sender.authenticate():
                print("❌ Gmail authentication failed.")
                return {'sent': 0, 'failed': 0}

        sent_count = 0
        failed_count = 0

        for result in self.results:
            if not result['emails_found']:
                print(f"⚠️ No emails found for {result['company']}, skipping...")
                failed_count += 1
                continue

            # Send to the first email found
            target_email = result['emails_found'][0]

            print(f"\n📤 Sending email to {result['company']} ({target_email})...")

            success = self.email_sender.send_email(
                to_email=target_email,
                subject=result['subject'],
                body=result['email_body'],
                from_name=self.profile.name
            )

            if success:
                result['sent'] = True
                result['sent_to'].append(target_email)
                sent_count += 1
                print(f"✅ Email sent successfully to {target_email}")
            else:
                failed_count += 1
                print(f"❌ Failed to send email to {target_email}")

            # Delay between sends
            time.sleep(random.uniform(3, 6))

        return {'sent': sent_count, 'failed': failed_count}

    def get_summary(self) -> str:
        """Get a summary of the email campaign"""
        if not self.results:
            return "No companies processed yet."

        total_companies = len(self.results)
        companies_with_emails = sum(1 for r in self.results if r['emails_found'])
        total_emails_found = sum(len(r['emails_found']) for r in self.results)
        emails_sent = sum(1 for r in self.results if r['sent'])

        summary = f"""
📊 Email Campaign Summary (Gemma 3n Powered):
- Companies processed: {total_companies}
- Companies with emails found: {companies_with_emails}
- Total email addresses discovered: {total_emails_found}
- Emails sent: {emails_sent}
- Success rate: {(companies_with_emails/total_companies)*100:.1f}%

📋 Detailed Results:
"""

        for result in self.results:
            emails_count = len(result['emails_found'])
            keywords = ', '.join(result['company_keywords'][:3]) if result['company_keywords'] else 'N/A'
            status = "✅ Sent" if result['sent'] else "📝 Generated"
            summary += f"- {result['company']}: {emails_count} emails found, Keywords: {keywords}, Status: {status}\n"

        return summary

print("✅ EmailAgent class defined successfully!")

✅ EmailAgent class defined successfully!


## 👤 Personal Profile Setup

In [ ]:
# Your updated profile based on the provided information
my_profile = PersonalProfile(
    name="Munib Ur Rehman Memon",
    email="munibmemon22@gmail.com",
    linkedin="www.linkedin.com/in/muniburrehman-memon-499862234",
    github="https://github.com/MunibUrRehmanMemon",
    skills=[
        "Machine Learning", "Deep Learning", "Computer Vision", "Python",
        "TensorFlow", "PyTorch", "RAG", "LLM fine-tuning", "Generative AI",
        "React Native", "FAISS", "Mistral", "Gemma", "Agentic RAG"
    ],
    projects=[
        "Fitro - AI-Powered Fitness App (Startup - NIC Hyderabad Cohort 7) using React Native, AI/ML",
        "Islamic Hadith Chatbot (RAG Implementation) using Python, FAISS, Mistral",
        "Financial Assistant Chatbot (Hackathon Winner) using Python, Gemma",
        "ICU readmission predictor to assist hospital management",
        "Real-world hospital dataset compilation with hundreds of patients' data"
    ],
    achievements=[
        "Runner-up at PROCOM '25- FAST NUCES (ML Category)",
        "Winner at Hacktober Hackathon (AWS & GitHub)",
        "Winner at Tech Arena in Progressive Programming",
        "Startup incubated in NIC Hyderabad Cohort 7",
        "Technical Lead - Google Developer Groups on Campus MUET",
        "General Secretary - ACM Student Chapter"
    ],
    goal="Find a role in AI/ML research or development that aligns with my skillset in Machine Learning, Computer Vision, and Generative AI."
)

print("✅ Personal profile created successfully!")
print(f"Profile: {my_profile.name}")
print(f"Skills: {my_profile.skills_str()[:100]}...")
print(f"Goal: {my_profile.goal}")

✅ Personal profile created successfully!
Profile: Munib Ur Rehman Memon
Skills: Machine Learning, Deep Learning, Computer Vision, Python, TensorFlow, PyTorch, RAG, LLM fine-tuning,...
Goal: Find a role in AI/ML research or development that aligns with my skillset in Machine Learning, Computer Vision, and Generative AI.


## 🎯 Target Companies Setup

In [ ]:
# Target companies (you can modify this list)
target_companies = {
    "OpenAI": "https://openai.com/careers",
    "Anthropic": "https://www.anthropic.com/careers",
    "Hugging Face": "https://huggingface.co/careers",
    "Stability AI": "https://stability.ai/careers",
    "Cohere": "https://cohere.com/careers"
}

print("✅ Target companies configured:")
for company, url in target_companies.items():
    print(f"- {company}: {url}")

✅ Target companies configured:
- OpenAI: https://openai.com/careers
- Anthropic: https://www.anthropic.com/careers
- Hugging Face: https://huggingface.co/careers
- Stability AI: https://stability.ai/careers
- Cohere: https://cohere.com/careers


## 🚀 Run Email Agent (Preview Mode)

In [ ]:
# Initialize and run the email agent
print("🤖 Initializing Email Agent with Gemma 3n E4B...")
agent = EmailAgent(profile=my_profile)

print("\n🚀 Starting Email Agent...")
print("⚠️  Running in PREVIEW mode. Emails will be generated but not sent.\n")

# Process companies and generate emails
results = agent.process_companies(
    companies=target_companies,
    send_emails=False  # Set to False for preview mode
)

# Print final summary
print("\n" + "="*60)
print(agent.get_summary())
print("="*60)

🤖 Initializing Email Agent with Gemma 3n E4B...
Using device: cuda

🚀 Starting Email Agent...
⚠️  Running in PREVIEW mode. Emails will be generated but not sent.

🔄 Loading Gemma 3n E4B model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

✅ Gemma 3n E4B model loaded successfully!

📧 Generated Email for OpenAI:
Company Keywords: 
Found Emails: hello@openai.com, hiring@openai.com, contact@openai.com
Subject: Regarding opportunities at OpenAI

Email Body:
Dear OpenAI Team,

I've been consistently impressed by OpenAI's groundbreaking work in Generative AI, particularly with models like Gemma. As a driven Machine Learning and Computer Vision enthusiast with experience in LLM fine-tuning and Agentic RAG, I'm deeply inspired by your mission to ensure AI benefits all of humanity.

My projects, including Fitro (an AI-powered fitness app using React Native) and an Islamic Hadith Chatbot leveraging Mistral and FAISS, demonstrate my passion for building practical AI solutions. I also have experience with RAG implementations and am actively exploring Generative AI applications.

I'd be grateful for the opportunity to briefly connect for an informational chat to learn more about your team's current work and gain insights into the AI 

## 📤 Send Generated Emails (Separate Cell)

In [ ]:
# Send the generated emails (run this cell only after reviewing the generated emails above)
print("📤 Sending generated emails...")
print("⚠️  Make sure you have reviewed the generated emails above before proceeding.\n")

# Uncomment the line below to actually send emails
# send_results = agent.send_generated_emails()

# For now, just show what would be sent
print("📋 Emails ready to send:")
for result in agent.results:
    if result['emails_found']:
        print(f"✉️ {result['company']}: {result['emails_found'][0]}")
    else:
        print(f"❌ {result['company']}: No email found")

print("\n💡 To actually send emails, uncomment the 'send_results = agent.send_generated_emails()' line above.")

## 📊 Final Summary and Export

In [ ]:
# Export results to a file for future reference
import json
from datetime import datetime

# Create export data
export_data = {
    "timestamp": datetime.now().isoformat(),
    "profile": {
        "name": my_profile.name,
        "email": my_profile.email,
        "skills": my_profile.skills,
        "goal": my_profile.goal
    },
    "results": agent.results,
    "summary": agent.get_summary()
}

# Save to file
filename = f"email_campaign_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Results exported to: {filename}")
print("\n📊 Final Campaign Summary:")
print(agent.get_summary())

print("\n🎉 Email Agent execution completed!")
print("\n📝 Next Steps:")
print("1. Review the generated emails above")
print("2. If satisfied, run the 'Send Generated Emails' cell")
print("3. Monitor your Gmail for responses")
print("4. Follow up appropriately with interested companies")